In [ ]:
#### FIRST PHASE #####

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)


In [ ]:
#https://www.kaggle.com/chicago/chicago-red-light-and-speed-camera-data
csv_path_locations= "Data_Files/speed-camera-locations.csv"
csv_path_violations="Data_Files/speed-camera-violations.csv"

#https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
csv_path_crashes="Data_Files/Traffic_Crashes_-_Crashes.csv"

camera_locations_df= pd.read_csv(csv_path_locations)
violations_df= pd.read_csv(csv_path_violations)
crashes_df= pd.read_csv(csv_path_crashes)
pd.set_option('max_columns', None)
violations_df


In [ ]:
#Filter out violation columns and pull out year 

violation_dates= violations_df[["CAMERA ID","VIOLATION DATE"]]
violation_dates['YEAR']= pd.DatetimeIndex(violation_dates["VIOLATION DATE"]).year
violation_dates['MONTH']= pd.DatetimeIndex(violation_dates["VIOLATION DATE"]).month
violation_dates.head()
violation_dates_grouped= violation_dates.groupby("YEAR").count()
violation_dates_grouped

In [ ]:
#Filter out camera location columns and pull out year
camera_locations_df.head()
# crash_dates_df['YEAR']= pd.DatetimeIndex(crash_dates_df["CRASH_DATE"]).year
camera_install_dates= camera_locations_df[["GO-LIVE DATE","ADDRESS"]]
camera_install_dates['YEAR']= pd.DatetimeIndex(camera_install_dates["GO-LIVE DATE"]).year
camera_install_dates['MONTH']= pd.DatetimeIndex(camera_install_dates["GO-LIVE DATE"]).month
camera_install_dates.head()

cameras_byYear= camera_install_dates.groupby("YEAR").count()
cameras_byYear.head()

#add missing years
missing_years= {"YEAR":[2016,2017], "GO-LIVE DATE":[0,0], "ADDRESS":[0,0], "MONTH":[0,0] }
missing_years_df= pd.DataFrame(missing_years)
missing_years_df= missing_years_df.set_index("YEAR")
missing_years_df

cameras_byYear= cameras_byYear.append(missing_years_df)
cameras_byYear= cameras_byYear.sort_index(ascending=True)
cameras_byYear

#add totals cameras at end of each year
years= cameras_byYear.index.tolist()
year_totals= []
total_installed= 0
for year in years:
    total_installed=  total_installed + cameras_byYear.loc[year, ["GO-LIVE DATE"]]
    year_totals.append(int(total_installed))
    
cameras_byYear["TOTAL LIVE"]= year_totals
cameras_byYear

In [ ]:
#Filter out crash date columns and pull out year

crash_dates_df= crashes_df[["CRASH_DATE","CRASH_MONTH","INJURIES_TOTAL"]]
crash_dates_df.head()
pd.set_option('mode.chained_assignment', None)
crash_dates_df['YEAR']= pd.DatetimeIndex(crash_dates_df["CRASH_DATE"]).year
crash_dates_df.head()
crash_dates_grouped= crash_dates_df.groupby("YEAR").count()

Displaying All Tables

In [ ]:
violation_dates_grouped

In [ ]:
cameras_byYear

In [ ]:
crash_dates_grouped

In [ ]:
violations_cameras_merge= pd.merge(violation_dates_grouped,cameras_byYear, on="YEAR", how="outer")
violations_cameras_merge

In [ ]:
violations_cameras_merge= violations_cameras_merge[["CAMERA ID","GO-LIVE DATE","TOTAL LIVE"]]
violations_cameras_merge= violations_cameras_merge.rename(columns= {"CAMERA ID":"TOTAL VIOLATIONS",
                                                                    "GO-LIVE DATE":"CAMERAS ADDED",
                                                                   "TOTAL LIVE":"TOTAL CAMERAS LIVE"})
violations_cameras_merge

In [ ]:
#delete rows 2013 and 2019 due to mismatch in data ranges
violations_cameras_merge.sort_index(ascending=True)
violation_cameras_clean= violations_cameras_merge.drop(labels=[2013,2019], axis=0, inplace=False)
violation_cameras_clean

In [ ]:
cameras_and_crashes= pd.merge(crash_dates_grouped,cameras_byYear, on="YEAR", how="outer")
cameras_and_crashes= cameras_and_crashes[["CRASH_DATE","GO-LIVE DATE","TOTAL LIVE"]]
cameras_and_crashes= cameras_and_crashes.rename(columns={"CRASH_DATE":"TOTAL CRASHES",
                                                         "GO-LIVE DATE":"CAMERAS ADDED",
                                                         "TOTAL LIVE":"TOTAL CAMERAS LIVE"})
cameras_and_crashes

In [ ]:
#delete rows 2013, 2019, 2020, and 2021 due to mismatch in data ranges
cameras_and_crashes_clean= cameras_and_crashes.drop(labels=[2013,2019,2020,2021], axis=0, inplace=False)


In [ ]:

# output merged dataframes to CSVs

cameras_and_crashes.to_csv('Output Data/cameras_and_crashes.csv', index=True)



cameras_and_crashes_clean.to_csv('Output Data/cameras_and_crashes_clean.csv', index=True)
violation_cameras_clean.to_csv('Output Data/violations_cameras_clean.csv', index=True)

In [ ]:
#### FIRST PHASE ENDS HERE #####